# Notebook - LangChain RAG Tutorial

**Escuela Colombiana de Ingeniería Julio Garavito**

**Student:** Santiago Botero García

## Step 0: Setup & Imports

This notebook demonstrates a **Retrieval-Augmented Generation (RAG)** pipeline using LangChain,
with support for **OpenAI** and **Google Gemini** as interchangeable LLM providers.


In [ ]:
%pip install langchain langchain-text-splitters langchain-community bs4 chromadb
import os
import getpass
import bs4

from langchain.agents import create_agent
from langchain.tools import tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

## Step 0: API Key Detection

The pipeline automatically detects which LLM provider is available
based on environment variables:

- `OPENAI_API_KEY` &rarr; OpenAI
- `GOOGLE_API_KEY` &rarr; Gemini


In [ ]:
LLM_PROVIDER = None

if os.getenv("OPENAI_API_KEY"):
    LLM_PROVIDER = "openai"
elif os.getenv("GOOGLE_API_KEY"):
    LLM_PROVIDER = "gemini"
else:
    raise RuntimeError("No API key found (OPENAI_API_KEY or GOOGLE_API_KEY required)")

print(f"[+] Using provider: {LLM_PROVIDER}")

## Step 1: Load and Chunk Documents

### Web Document Loading

External web content is loaded and split into overlapping chunks
to optimize embedding quality and semantic retrieval.

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

### Loading Documents into Memory

The webpage content is loaded and normalized into LangChain’s `Document` format, which includes:

* `page_content` (the text)
* `metadata` (source URL and other attributes)

This abstraction allows downstream components (splitters, vector stores, retrievers) to work consistently across different data sources.

In [ ]:
docs = loader.load()
print(f"[+] Loaded {len(docs)} docs — characters: {len(docs[0].page_content)}")

### Text Chunking

Large documents are split into smaller overlapping chunks to optimize retrieval and embedding quality.

* `chunk_size=1000` ensures each chunk fits comfortably within model context limits.
* `chunk_overlap=200` preserves semantic continuity between chunks.
* The recursive strategy attempts to split on natural boundaries (paragraphs, sentences) when possible.

This step reflects the RAG principle that **retrieval happens at the chunk level, not the document level**.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

splits = text_splitter.split_documents(docs)
print(f"[+] Split into {len(all_splits)} chunks")

## Step 2: Vector Store & Retrieval Setup

This section introduces the components responsible for semantic search:

* `OpenAIEmbeddings` converts text chunks into high-dimensional vectors.
* `Chroma` is a lightweight, local vector database used to store and search those embeddings.

LangChain’s RAG tutorial highlights that vector stores are **pluggable**, and Chroma is used here for simplicity and fast prototyping.

In [ ]:
if LLM_PROVIDER == "openai":
    from langchain_openai import ChatOpenAI, OpenAIEmbeddings

    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-large"
    )

    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0
    )

elif LLM_PROVIDER == "gemini":
    from langchain_google_genai import (
        ChatGoogleGenerativeAI,
        GoogleGenerativeAIEmbeddings
    )

    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001"
    )

    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0
    )

### Creating Embeddings and Vector Store

* Each text chunk is embedded using an OpenAI embedding model.
* The embeddings are stored in Chroma, enabling similarity-based retrieval.
* This forms the knowledge base that the RAG system will query at runtime.

At this point, the pipeline has transformed raw web content into a searchable semantic index.

In [ ]:
vector_store = Chroma.from_documents(
    splits,
    embedding=embeddings
)

print("[+] Vector store created")

## Step 3: RAG Tool for the Agent

Retrieval is exposed as a LangChain tool so the agent can explicitly
query the vector store when needed.


In [ ]:
@tool
def retrieve_context(query: str) -> str:
    """Retrieve relevant document chunks from the vector store."""
    docs = vector_store.similarity_search(query, k=4)
    return "\n\n".join(d.page_content for d in docs)

## Step 4: Create the Agent

The system prompt defines the agent’s role and behavior.

* It explicitly instructs the agent to rely on retrieval rather than hallucation.
* This aligns with LangChain’s recommendation to **steer agent behavior via prompts**, not just code.

In [ ]:
system_prompt = (
    "You are a retrieval-augmented generation agent. "
    "Use the provided retrieval tool to answer user queries."
)

This creates a LangChain agent that:

* Uses the default chat model configured via environment variables
* Has access to the retrieval tool
* Can reason about when to call the tool versus when to answer directly

This is the core orchestration layer of the RAG system.

In [ ]:
agent = create_openai_tools_agent(
    llm=llm,
    tools=[retrieve_context],
    system_prompt=system_prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=[retrieve_context],
    verbose=True
)

print("[+] Agent created")

## Step 5: Running the Agent

A sample user query is provided to demonstrate the full RAG loop:

1. The agent receives the question
2. It invokes the retrieval tool
3. Retrieved context is injected
4. The final answer is generated

In [ ]:
query = "What is task decomposition?"

The agent is executed in streaming mode, allowing you to observe intermediate reasoning and tool usage step by step.

This reflects the LangChain tutorial’s focus on **inspectability and transparency**, making RAG systems easier to debug and understand.

In [ ]:
print(f"\nQuery: {query}\n")
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()